In [ ]:
import geopandas as gpd
import pandas as pd
from pyproj import Proj, CRS, Transformer
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
from folium import plugins
import branca
import branca.colormap as cm
import seaborn as sns
import warnings
plt.rcParams['figure.figsize'] = [8,8]



In [ ]:
gb_ne = CRS('epsg:29901')
ll = CRS('epsg:4326')
tr = Transformer.from_crs(gb_ne, ll)

In [ ]:
df = pd.read_csv("../csv/soil/regionalsoilsaaquaregiahdl.csv")
gdf = gpd.GeoDataFrame(df)
gdf

In [ ]:
gdf["Sample"].nunique()

In [ ]:
def map_row(row):
  latlng = tr.transform(row["Easting"], row["Northing"])
  return Point(latlng[1], latlng[0])

#def lat_lng(data):
#  lat,lng = tr.transform(data["Easting"], data["Northing"])
#  return (lat, lng)

In [ ]:
geometry = pd.DataFrame(gdf.apply(map_row, axis=1), columns = ["geometry"])

#latlng = pd.DataFrame(lat_lng(gdf), index = ["lat", "lng"]).T

In [ ]:
#aquaRD = pd.concat([gdf, latlng, geometry], axis=1)
aquaRD = pd.concat([gdf, geometry], axis=1)
aquaRD.set_geometry("geometry", inplace=True)
aquaRD

In [ ]:
#aquaRD_df = aquaRD[["Sample","P","Se","Cu","Mo","Mn","Ca","K","Zn","Mg","Pb","Cd","As","lat","lng","geometry"]]
aquaRD_df = aquaRD[["Sample","P","Se","Cu","Mo","Mn","Ca","K","Zn","Mg", "Na","Pb","Cd","As", "Ni","Cr","Hg","geometry"]]
#aquaRD_df

In [ ]:
xrf = pd.read_csv("../csv/soil/regionalsoilsaxrf.csv")
xrf_df= pd.DataFrame(xrf[["Sample","pH", "LOI"]])

In [ ]:
ard_a = pd.merge(aquaRD_df, xrf_df, on='Sample', how='inner')

In [ ]:
#def som(row):
#    return round(100-row["LOI"],2)

#ard_a['%OM'] = ard_a.apply(som,axis=1)
#ard_a

ard_a = ard_a.rename(columns={'LOI':'%OM'})
ard_a

In [ ]:
ard_a.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

In [ ]:
a = pd.to_numeric(ard_a["P"], errors='coerce')
idx=a.isna()
ard_a[idx]

In [ ]:
ard_A=ard_a.drop([599,615])

In [ ]:
ard_A.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

In [ ]:
a = pd.to_numeric(ard_A["Pb"], errors='coerce')
idx=a.isna()
ard_A[idx]

In [ ]:
ard_A["Mn"]=ard_A["Mn"].replace("--", 0)
ard_A["Mn"]=ard_A["Mn"].replace(">10000", 10100)
ard_A["Pb"]=ard_A["Pb"].replace(">10000", 10100)


In [ ]:
ard_A.to_csv("surface_soil_aqua.csv", index=False)

In [ ]:
ard_A.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

In [ ]:
ard_A

In [ ]:
sum(i>3 for i in pd.to_numeric(ard_A["Se"]))

In [ ]:
coordinates = pd.DataFrame()
coordinates['lat']=ard_A.geometry.y
coordinates['lng']=ard_A.geometry.x

In [ ]:
def create_nimap():
    nimap = folium.Map(
        location=[54.637521, -6.485935],
        zoom_start=9
    )
    return nimap

In [ ]:
ni_map = create_nimap()
plugins.MarkerCluster(coordinates[['lat','lng']]).add_to(ni_map)

fig = folium.Figure(width=1500, height=1000)
ni_map.add_to(fig)

In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1300, height=800)
colormap = cm.linear.viridis.to_step(index=range(0,9,1))

pH=ard_A.explore("pH", m=ni_map, cmap = colormap, marker_kwds={"radius": 3}, name = "Results")

folium.TileLayer('stamenterrain').add_to(pH)
folium.TileLayer('cartodbpositron').add_to(pH)
folium.TileLayer('cartodbdark_matter').add_to(pH)
folium.LayerControl().add_to(pH)


pH.add_to(fig)

In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1300, height=1000)
colormap = cm.linear.viridis.to_step(index=range(0,110,20))
colormap.caption="%OM"
                                                  
OM=ard_A.explore("%OM", cmap=colormap, m=ni_map, marker_kwds={"radius": 3}, name="Results")

folium.TileLayer('stamenterrain').add_to(OM)
folium.TileLayer('cartodbpositron').add_to(OM)
folium.TileLayer('cartodbdark_matter').add_to(OM)
folium.LayerControl().add_to(OM)
OM.add_to(fig)

In [ ]:
ph_om_summary=pd.DataFrame()
ph_om_summary["pH"]=round(ard_A["pH"].describe(),2)
ph_om_summary["%OM"]=round(ard_A["%OM"].describe(),2)
ph_om_summary.iloc[0].astype(int)
ph_om_summary

In [ ]:
#ard_A["pH"].plot.kde(bw_method=0.1)

In [ ]:
#ard_A["pH"]
#ard_A["%OM"]

In [ ]:
fig, axes = plt.subplots(1, 2,  figsize=(15, 5),sharex=False, sharey=False)
fig.suptitle('$\\bf{Distribution\ plots\ for\ pH\ and\ \%Organic\ Matter}$')
sns.set_style("darkgrid")


# Bulbasaur
sns.kdeplot(ax=axes[0], x=ard_A["pH"], fill=True)
axes[0].set_title("pH")
axes[0].set(xlabel=None)

# Charmander
sns.kdeplot(ax=axes[1], x=ard_A["%OM"], fill=True, color='crimson')
axes[1].set_title("%OM")
axes[1].set(xlabel=None)
warnings.filterwarnings('ignore')

In [ ]:
#ard_A["%OM"].plot.kde(bw_method=0.05)

In [ ]:
print()
print("Low pH AND low Organic matter")
print()
ll=ard_A.loc[(ard_A['pH']<= 3.5) & (ard_a['%OM']<20)].copy()
ll

In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1500, height=1000)

ll_coord = pd.DataFrame()
ll_coord['lat']=ll.geometry.y
ll_coord['lng']=ll.geometry.x

plugins.MarkerCluster(ll_coord[['lat','lng']]).add_to(ni_map)
ni_map.add_to(fig)

In [ ]:
basic_cations=(ard_A[["Ca","Mg","K","Na"]]).copy().astype(float)

def basic_conversion(data):
    Ca_=(data["Ca"])/200
    Mg_=(data["Mg"])/120
    K_=(data["K"])/390
    Na_=(data["Na"])/23
    return [Ca_, Mg_, K_, Na_]

basic_meq = pd.DataFrame(basic_conversion(basic_cations), index = ["Ca","Mg","K","Na"]).T
basic_meq

In [ ]:
acid_cations=pd.DataFrame(ard_A["pH"]).copy()

In [ ]:
acid_cations["pH_eq"]=acid_cations["pH"].apply(lambda ph: 12 * (7 - ph) if ph < 7 else 0)
#acid_cations

In [ ]:
CEC_df = pd.concat([basic_meq, acid_cations["pH_eq"]], axis=1)
CEC_df["CEC"]=round(CEC_df.sum(axis=1),2)

#CEC_df

In [ ]:
CEC_saturation = CEC_df["CEC"].copy().to_frame()

CEC_saturation["Acid%"] = round((CEC_df["pH_eq"]/CEC_df["CEC"])*100,2)
CEC_saturation["Base%"] = round(((CEC_df["Ca"]+CEC_df["Mg"]+CEC_df["K"]+CEC_df["Na"])/CEC_df["CEC"])*100,2)
CEC_saturation["Ca%"] = round((CEC_df["Ca"]/CEC_df["CEC"])*100,2)
CEC_saturation["Mg%"] = round((CEC_df["Mg"]/CEC_df["CEC"])*100,2)
CEC_saturation["K%"] = round((CEC_df["K"]/CEC_df["CEC"])*100,2)
CEC_saturation["Na%"] = round((CEC_df["Na"]/CEC_df["CEC"])*100,2)

In [ ]:
#CEC_saturation

In [ ]:
def soil_texture(data):
    if data <=9:
        return "Sand"
    elif data >10 and data <=17:
        return "Loam"
    elif data >17 and data <=25:
        return "Silt"
    elif data >25 and data<=50:
        return "Clay"
    else:
        return "Organic Soil"

In [ ]:
CEC_saturation["Type"] = CEC_saturation["CEC"].apply(soil_texture)

In [ ]:
CEC_saturation

In [ ]:
soil_type=CEC_saturation.copy().groupby(["Type"])["Type"].count().nlargest().to_frame()
print(soil_type)
plt.pie(soil_type["Type"], labels=soil_type.index, radius=1);

In [ ]:
CEC_loc = gpd.GeoDataFrame(pd.concat([CEC_saturation, ard_A.geometry], axis =1))
CEC_loc.head()

In [ ]:
soil_type = CEC_loc.copy()
soil_type['lat']=soil_type.geometry.y
soil_type['lng']=soil_type.geometry.x

Org_soil = soil_type[(soil_type['Type']=="Organic Soil")]
Clay_soil = soil_type[(soil_type['Type']=="Clay")]
Silt_soil = soil_type[(soil_type['Type']=="Silt")]
Loam_soil = soil_type[(soil_type['Type']=="Loam")]
Sand_soil = soil_type[(soil_type['Type']=="Sand")]


In [ ]:
from folium.plugins import GroupedLayerControl

m = create_nimap()
fig = folium.Figure(width=1200, height=1000)

group_1 = folium.FeatureGroup("Organic (Green)").add_to(m)
for loc, p in zip(zip(Org_soil["lat"],Org_soil["lng"]),Org_soil["Type"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          color='green',
                          fill_opacity=1
                          ).add_to(group_1)

group_2 = folium.FeatureGroup("Clay (Grey)").add_to(m)
for loc, p in zip(zip(Clay_soil["lat"],Clay_soil["lng"]),Clay_soil["Type"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          color='darkBrown',
                          fill_opacity=0.4
                          ).add_to(group_2)

group_3 = folium.FeatureGroup("Silt (Blue)").add_to(m)
for loc, p in zip(zip(Silt_soil["lat"],Silt_soil["lng"]),Silt_soil["Type"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          fill_opacity=1,
                          color='Blue').add_to(group_3)
      
group_4 = folium.FeatureGroup("Loam (Red)").add_to(m)
for loc, p in zip(zip(Loam_soil["lat"],Loam_soil["lng"]),Loam_soil["Type"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          fill_opacity=1,
                          color='red').add_to(group_4)
      
group_5 = folium.FeatureGroup("Sand (Yellow)").add_to(m)
for loc, p in zip(zip(Sand_soil["lat"],Sand_soil["lng"]),Sand_soil["Type"]):
      folium.CircleMarker(location=loc,
                          radius= 3,
                          fill=True,
                          fill_opacity=1,
                          color='Yellow').add_to(group_5)
      

folium.TileLayer('stamenterrain', name="Terrain").add_to(m)
folium.TileLayer('cartodbpositron', name="Greyscale").add_to(m)
folium.TileLayer('cartodbdark_matter', name="Dark").add_to(m)
plugins.Geocoder().add_to(m)
folium.LayerControl(collapsed = False).add_to(m)
'''
GroupedLayerControl(
    groups={
      'LEGEND': [group_1, group_2, group_3, group_4, group_5]
    },
    exclusive_groups=False,
    collapsed=False
).add_to(m)
'''

m.add_to(fig)

In [ ]:
'''colours = colors=['silver','red','green','yellow', 'blue']

ni_map = create_nimap()

CEC_map = CEC_loc.explore(
    m=ni_map,
    column="Type",
    tiles="Stamen Terrain",
    cmap=colours,
    marker_kwds=dict(radius=3),
    name="Results"
)

folium.TileLayer('stamenterrain').add_to(CEC_map)
folium.LayerControl().add_to(CEC_map)

CEC_map''';

In [ ]:
ardA_cec = pd.concat([ard_A, CEC_saturation["Type"]], axis=1)
ardA_cec.to_csv("surface_soil_CEC.csv", index=False)
ardA_cec.drop(["Sample","geometry"], axis=1, inplace=True)
ardA_cec

In [ ]:
ardA_corr=ardA_cec.replace(['Sand','Loam','Silt','Clay','Organic Soil'], [1,2,3,4,5])
ardA_corr=ardA_corr.apply(pd.to_numeric, errors='coerce')
ardA_Corr=ardA_corr.corr().round(2)

In [ ]:
sns.set (rc = {'figure.figsize':(30, 10)})
sns.heatmap(ardA_Corr,  cmap="BrBG", fmt='g',vmin=-1, vmax=1, annot=True).set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)

In [ ]:
P=gpd.GeoDataFrame(pd.concat([(pd.to_numeric(ard_A["P"], errors='coerce')),ard_A["pH"], ard_A["geometry"]], axis=1))
P

In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1200, height=1000)
colormap = cm.linear.Accent_06.to_step(index=[0,1500,9000])

P_map=P.explore("P", cmap = colormap, m=ni_map, marker_kwds={"radius": 3})
P_map.add_to(fig)

In [ ]:
bad_ph=((ard_A["pH"].lt(6.2).sum())/len(ard_A))*100
print("Number of samples less than lower threshold of healthy pH (6.2) = ", ard_A["pH"].lt(6.2).sum(), " = ", round(bad_ph),"%")


In [ ]:
ni_map = create_nimap()
fig = folium.Figure(width=1200, height=1000)
colormap = cm.LinearColormap(colors=["salmon","navy"]).to_step(index=[0,6,8])

pH_map=ard_A.explore(column="pH", cmap = colormap, m=ni_map, marker_kwds={"radius": 3})
pH_map.add_to(fig)

In [ ]:
ni_map = create_nimap()
fig = folium.Figure(1300,2000)

################### pH MATTER MAP ######################


ph_colormap = cm.linear.viridis.to_step(index=range(0,9,1))
ph_colormap.caption="pH"

pH=folium.Map(
        location=[54.637521, -6.485935],
        zoom_start=8.5,
        position="absolute",
        left="0%",
        height="50%")

ard_A.explore("pH", m=pH, cmap = ph_colormap, marker_kwds={"radius": 3}, name = "pH")

folium.TileLayer('stamenterrain').add_to(pH)
folium.TileLayer('cartodbpositron').add_to(pH)
folium.TileLayer('cartodbdark_matter').add_to(pH)
folium.LayerControl(collapsed=False).add_to(pH)


################### ORGANIC MATTER MAP ######################

OM=folium.Map(
        location=[54.637521, -6.485935],
        zoom_start=8.5,
        position="absolute",
        left="0%",
        height="50%",
        top="50%")
OM_colormap = cm.linear.viridis.to_step(index=range(0,110,20))
OM_colormap.caption="%OM"
                                                  
ard_A.explore("%OM", cmap=OM_colormap, m=OM, marker_kwds={"radius": 3}, name="%OM")

folium.TileLayer('stamenterrain').add_to(OM)
folium.TileLayer('cartodbpositron').add_to(OM)
folium.TileLayer('cartodbdark_matter').add_to(OM)
folium.LayerControl(collapsed=False).add_to(OM)

fig.add_child(pH)
fig.add_child(OM)